In [ ]:
import os
import glob
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from datetime import datetime
import math
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from utils import *
%matplotlib inline

In [ ]:
def get_30min_wise_data(day_data):
    '''Returns time, day and load values for each 30mins of day_data in a numpy matrix'''
    data = []
    for i in range(0, 288, 6):
#         time = day_data[i][0]
        load = day_data[i][1]
        day = day_data[i][2].weekday()
        data.append([i, day, load])
    return np.asarray(data)

In [ ]:
def get_date_data(date_path):
    '''read csv data at date_path and return it in a numpy matrix'''
    temp = pd.read_csv(date_path)
    date_ = date_path.split('/')[-1][:-4] # get date of '01-10-2017' format
    temp['date'] = datetime.strptime(date_, "%d-%m-%Y").date()
    temp = extract_dt(temp)
    return temp.as_matrix()

In [ ]:
def get_data():
    '''Return data of all csv files in SLDC_Data/ folder in matrix form'''
    data = []
    for date_path in glob.glob('SLDC_Data/*/*/*.csv'):
        try:
            date_data = get_date_data(date_path)
            temp = get_30min_wise_data(date_data)
            data.append(temp)
        except Exception as e:
            print(date_path, e)
    data = np.asarray(data)
    data = data.reshape(-1, 3)
    np.random.shuffle(data)
    return data

In [ ]:
# data preprocessing
data = get_data()
X = data[:, :-1] # features
Y = data[:, -1].reshape(-1, 1) # load, reshaped to 2D

In [ ]:
# rescale the data to 0-1 values
feature_scaler = MinMaxScaler(feature_range=(0, 1))
load_scaler = MinMaxScaler(feature_range=(0, 1)) #StandardScaler()
X = feature_scaler.fit_transform(X)
Y = load_scaler.fit_transform(Y)

In [ ]:
X.shape, Y.shape

In [ ]:
def getModel():
    model = Sequential()
    model.add(Dense(32, input_shape=(2, )))
    model.add(Activation('relu'))
    model.add(Dense(32, input_shape=(2, )))
    model.add(Activation('relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='mse', optimizer='adam')
    print(model.summary())
    return model

In [ ]:
model = getModel()

In [ ]:
model.fit(X, Y, batch_size=32, epochs=50, verbose=1, validation_split=0.2, shuffle=True)

In [ ]:
preds = model.predict(X)
preds = load_scaler.inverse_transform(preds)
Y_ = load_scaler.inverse_transform(Y)

In [ ]:
x = np.random.randint(len(preds))
print(x,x+500)
plt.plot(range(len(preds[x:x+500])), preds[x:x+500])
plt.plot(range(len(Y_[x:x+500])), Y_[x:x+500])
plt.legend(['preds', 'Y_'])
plt.show()